# Recuperación BDD

## Nombre: Joel Molina

## NRC: 8903

## Librerías

In [69]:
import pandas as pd 
import numpy as np 
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import pyodbc
import sqlite3
from datetime import datetime
import csv
import random

## Leer archivo Building_Permits

In [2]:
DataSet = pd.read_csv('Building_Permits.csv', low_memory=False) 

In [3]:
DataSet

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198895,M862628,8,otc alterations permit,12/05/2017,0113,017A,1228,NaN,Montgomery,St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1489337276729
198896,201712055595,8,otc alterations permit,12/05/2017,0271,014,580,NaN,Bush,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,NaN,NaN,NaN,NaN,1489462354993
198897,M863507,8,otc alterations permit,12/06/2017,4318,019,1568,NaN,Indiana,St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1489539379952
198898,M863747,8,otc alterations permit,12/06/2017,0298,029,795,NaN,Sutter,St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1489608233656


In [17]:
DataSet['Location']

0         (37.785719256680785, -122.40852313194863)
1          (37.78733980600732, -122.41063199757738)
2           (37.7946573324287, -122.42232562979227)
3          (37.79595867909168, -122.41557405519474)
4          (37.78315261897309, -122.40950883997789)
                            ...                    
198895                                          NaN
198896                                          NaN
198897                                          NaN
198898                                          NaN
198899                                          NaN
Name: Location, Length: 198900, dtype: object

## Conexión con SQL Server

In [23]:
conec = sqlalchemy.create_engine('mssql+pyodbc://JOELMOLINA/BD_Transaccional_Building_Permits?driver=SQL+Server+Native+Client+11.0')

## Contar los datos faltantes del archivo Building_Permits

In [5]:
DatosFaltantes = DataSet.isnull().sum()
DatosFaltantes

Permit Number                                  0
Permit Type                                    0
Permit Type Definition                         0
Permit Creation Date                           0
Block                                          0
Lot                                            0
Street Number                                  0
Street Number Suffix                      196684
Street Name                                    0
Street Suffix                               2768
Unit                                      169421
Unit Suffix                               196939
Description                                  290
Current Status                                 0
Current Status Date                            0
Filed Date                                     0
Issued Date                                14940
Completed Date                            101709
First Construction Document Date           14946
Structural Notification                   191978
Number of Existing S

## Eliminación de las columnas con mas del 50 % de datos nulos

In [6]:
descartarDatosF= DataSet.drop(['Street Number Suffix', 'Unit', 'Unit Suffix', 'Completed Date', 'Structural Notification', 'Voluntary Soft-Story Retrofit', 'Fire Only Permit', 'TIDF Compliance','Site Permit'], axis=1)
descartarDatosF.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Street Suffix,Description,...,Plansets,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,St,"ground fl facade: to erect illuminated, electr...",...,2.0,3.0,constr type 3,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,St,remove (e) awning and associated signs.,...,2.0,3.0,constr type 3,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,Av,installation of separating wall,...,2.0,1.0,constr type 1,1.0,constr type 1,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,Av,repair dryrot & stucco at front of bldg.,...,2.0,5.0,wood frame (5),5.0,wood frame (5),3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,St,demolish retail/office/commercial 3-story buil...,...,2.0,3.0,constr type 3,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


### Columnas del DataSet resultantes de la eliminación

In [7]:
print("Columnas resultantes: %d" % descartarDatosF.shape[1])

Columnas resultantes: 34


## Transformar los datos Nulos 

In [8]:
DatosTransformados=descartarDatosF.fillna(method ='ffill')

In [9]:
DatosTransformados

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Street Suffix,Description,...,Plansets,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,St,"ground fl facade: to erect illuminated, electr...",...,2.0,3.0,constr type 3,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,St,remove (e) awning and associated signs.,...,2.0,3.0,constr type 3,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,Av,installation of separating wall,...,2.0,1.0,constr type 1,1.0,constr type 1,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,Av,repair dryrot & stucco at front of bldg.,...,2.0,5.0,wood frame (5),5.0,wood frame (5),3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,St,demolish retail/office/commercial 3-story buil...,...,2.0,3.0,constr type 3,5.0,wood frame (5),6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198895,M862628,8,otc alterations permit,12/05/2017,0113,017A,1228,Montgomery,St,street space,...,0.0,5.0,wood frame (5),5.0,wood frame (5),4.0,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)",1489337276729
198896,201712055595,8,otc alterations permit,12/05/2017,0271,014,580,Bush,St,fire alarm upgrade ref 201704123852,...,2.0,5.0,wood frame (5),5.0,wood frame (5),4.0,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)",1489462354993
198897,M863507,8,otc alterations permit,12/06/2017,4318,019,1568,Indiana,St,street space,...,2.0,5.0,wood frame (5),5.0,wood frame (5),4.0,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)",1489539379952
198898,M863747,8,otc alterations permit,12/06/2017,0298,029,795,Sutter,St,street space permit,...,2.0,5.0,wood frame (5),5.0,wood frame (5),4.0,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)",1489608233656


## Cargar los datos 

### DataFrameLocation

In [24]:

dataFrameLocation = pd.DataFrame({'Block': DatosTransformados['Block'], 'Lot' : DatosTransformados['Lot'], 'Street_Name': DatosTransformados['Street Name'], 
                     'Street_Suffix': DatosTransformados['Street Suffix'], 'Street_Number': DatosTransformados['Street Number'], 
                    'Neighborhoods_Analysis': DatosTransformados['Neighborhoods - Analysis Boundaries'], 'Zipcode': DatosTransformados['Zipcode'], 'Location': DatosTransformados['Location']  })
dataFrameLocation

,Block,Lot,Street_Name,Street_Suffix,Street_Number,Neighborhoods_Analysis,Zipcode,Location
0,0326,023,Ellis,St,140,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)"
1,0306,007,Geary,St,440,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)"
2,0595,203,Pacific,Av,1647,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)"
3,0156,011,Pacific,Av,1230,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)"
4,0342,001,Market,St,950,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)"
...,...,...,...,...,...,...,...,...
198895,0113,017A,Montgomery,St,1228,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)"
198896,0271,014,Bush,St,580,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)"
198897,4318,019,Indiana,St,1568,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)"
198898,0298,029,Sutter,St,795,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)"


In [30]:
dataFrameLocation.to_sql(name='location', con=conec, if_exists='append',index=False)

#### Extraigo el id_Location de la tabla location de la base de datos BD_Transaccional_Building_Permits para así poder ponerlo en la tabla Building

In [33]:
location=pd.read_sql_query("""SELECT id_Location From Location ; """ , con=conec.connect())
location

,id_Location
0,1
1,2
2,3
3,4
4,5
...,...
198895,198896
198896,198897
198897,198898
198898,198899


### DataFrameBuilding (Se agrega el id_Location extraido de la tabla anteriormente)

In [35]:
dataFrameBuilding = pd.DataFrame({'id_Location': location['id_Location'] ,'Record_ID': DatosTransformados['Record ID'], 'Description' : DatosTransformados['Description'], 'Current_Status': DatosTransformados['Current Status'], 
                     'Current_Status_Date': DatosTransformados['Current Status Date'], 'Field_Date': DatosTransformados['Filed Date'], 
                    'Issued_Date': DatosTransformados['Issued Date'], 'First_Construction_Document_Date': DatosTransformados['First Construction Document Date'], 'Estimated_Cost': DatosTransformados['Estimated Cost'],
                    'Revised_Cost': DatosTransformados['Revised Cost'], 'Plansets' : DatosTransformados['Plansets'], 'Existing_Construction_Type': DatosTransformados['Existing Construction Type'],
                    'Existing_Construction_Type_Description': DatosTransformados['Existing Construction Type Description'], 'Proposed_Construction_Type': DatosTransformados['Proposed Construction Type'], 'Proposed_Construction_Type_Description': DatosTransformados['Proposed Construction Type Description'],
                    'Supervisor_District': DatosTransformados['Supervisor District'], 'Number_of_Existing_Stories': DatosTransformados['Number of Existing Stories'], 'Number_of_Proposed_Stories': DatosTransformados['Number of Proposed Stories'],'Existing_Use': DatosTransformados['Existing Use'], 'Existing_Units':  DatosTransformados['Existing Units'],
                    'Proposed_Use': DatosTransformados['Proposed Use'], 'Proposed_Units': DatosTransformados['Proposed Units'] })
dataFrameBuilding

,id_Location,Record_ID,Description,Current_Status,Current_Status_Date,Field_Date,Issued_Date,First_Construction_Document_Date,Estimated_Cost,Revised_Cost,...,Existing_Construction_Type_Description,Proposed_Construction_Type,Proposed_Construction_Type_Description,Supervisor_District,Number_of_Existing_Stories,Number_of_Proposed_Stories,Existing_Use,Existing_Units,Proposed_Use,Proposed_Units
0,1,1380611233945,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,11/09/2015,4000.0,4000.0,...,constr type 3,NaN,NaN,3.0,6.0,NaN,tourist hotel/motel,143.0,NaN,NaN
1,2,1420164406718,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,08/03/2017,1.0,500.0,...,constr type 3,NaN,NaN,3.0,7.0,NaN,tourist hotel/motel,143.0,NaN,NaN
2,3,1424856504716,installation of separating wall,withdrawn,09/26/2017,05/27/2016,08/03/2017,08/03/2017,20000.0,500.0,...,constr type 1,1.0,constr type 1,3.0,6.0,6.0,retail sales,39.0,retail sales,39.0
3,4,1443574295566,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,07/18/2017,2000.0,2000.0,...,wood frame (5),5.0,wood frame (5),3.0,2.0,2.0,1 family dwelling,1.0,1 family dwelling,1.0
4,5,144548169992,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,11/20/2017,100000.0,100000.0,...,constr type 3,5.0,wood frame (5),6.0,3.0,2.0,retail sales,1.0,1 family dwelling,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198895,198896,1489337276729,street space,issued,12/05/2017,12/05/2017,12/05/2017,12/05/2017,7400.0,1.0,...,wood frame (5),5.0,wood frame (5),4.0,2.0,2.0,2 family dwelling,2.0,2 family dwelling,2.0
198896,198897,1489462354993,fire alarm upgrade ref 201704123852,issued,12/06/2017,12/05/2017,12/06/2017,12/06/2017,5000.0,5000.0,...,wood frame (5),5.0,wood frame (5),4.0,4.0,4.0,apartments,4.0,apartments,4.0
198897,198898,1489539379952,street space,issued,12/06/2017,12/06/2017,12/06/2017,12/06/2017,5000.0,1.0,...,wood frame (5),5.0,wood frame (5),4.0,4.0,4.0,apartments,4.0,apartments,4.0
198898,198899,1489608233656,street space permit,issued,12/06/2017,12/06/2017,12/06/2017,12/06/2017,5000.0,1.0,...,wood frame (5),5.0,wood frame (5),4.0,4.0,4.0,apartments,4.0,apartments,4.0


In [38]:
dataFrameBuilding.to_sql(name='Building', con=conec, if_exists='append',index=False)

####  Extraigo el id_Building de la tabla Building de la base de datos BD_Transaccional_Building_Permits para así poder ponerlo en la tabla Permit

In [39]:
Building=pd.read_sql_query("""SELECT id_Building From Building ; """ , con=conec.connect())
Building

,id_Building
0,1
1,2
2,3
3,4
4,5
...,...
198895,198896
198896,198897
198897,198898
198898,198899


### DataFramePermit

In [41]:

dataFramePermit = pd.DataFrame({'id_Building':Building['id_Building'], 'Permit_Number': DatosTransformados['Permit Number'], 'Permit_Type' : DatosTransformados['Permit Type'], 'Permit_Type_Definition': DatosTransformados['Permit Type Definition'], 
                     'Permit_Creation_Date': DatosTransformados['Permit Creation Date'], 'Permit_Expiration_Date': DatosTransformados['Permit Expiration Date']})
dataFramePermit

,id_Building,Permit_Number,Permit_Type,Permit_Type_Definition,Permit_Creation_Date,Permit_Expiration_Date
0,1,201505065519,4,sign - erect,05/06/2015,11/03/2016
1,2,201604195146,4,sign - erect,04/19/2016,12/03/2017
2,3,201605278609,3,additions alterations or repairs,05/27/2016,12/03/2017
3,4,201611072166,8,otc alterations permit,11/07/2016,07/13/2018
4,5,201611283529,6,demolitions,11/28/2016,12/01/2018
...,...,...,...,...,...,...
198895,198896,M862628,8,otc alterations permit,12/05/2017,06/05/2018
198896,198897,201712055595,8,otc alterations permit,12/05/2017,04/06/2018
198897,198898,M863507,8,otc alterations permit,12/06/2017,04/06/2018
198898,198899,M863747,8,otc alterations permit,12/06/2017,04/06/2018


In [42]:
dataFramePermit.to_sql(name='Permit', con=conec, if_exists='append',index=False)

# DATA WAREHOUSE

## Conexión

In [48]:
conecDW = sqlalchemy.create_engine('mssql+pyodbc://JOELMOLINA/DWH_EsquemaEstrella_BuildingPermits?driver=SQL+Server+Native+Client+11.0')

## Extracción de datos de la base de datos relacional BD_Transaccional_Building_Permits

In [44]:
df_Building=pd.read_sql_query("""SELECT Description, Current_Status, Plansets, Supervisor_District, Number_of_Existing_Stories, Number_of_Proposed_Stories From Building ; """ , con=conec.connect())
df_Building

,Description,Current_Status,Plansets,Supervisor_District,Number_of_Existing_Stories,Number_of_Proposed_Stories
0,"ground fl facade: to erect illuminated, electr...",expired,2.0,3.0,6.0,NaN
1,remove (e) awning and associated signs.,issued,2.0,3.0,7.0,NaN
2,installation of separating wall,withdrawn,2.0,3.0,6.0,6.0
3,repair dryrot & stucco at front of bldg.,complete,2.0,3.0,2.0,2.0
4,demolish retail/office/commercial 3-story buil...,issued,2.0,6.0,3.0,2.0
...,...,...,...,...,...,...
198895,street space,issued,0.0,4.0,2.0,2.0
198896,fire alarm upgrade ref 201704123852,issued,2.0,4.0,4.0,4.0
198897,street space,issued,2.0,4.0,4.0,4.0
198898,street space permit,issued,2.0,4.0,4.0,4.0


In [45]:
df_Location=pd.read_sql_query("""SELECT Block, Lot, Street_Name, Street_Suffix, Neighborhoods_Analysis, Zipcode, Location From Location ; """ , con=conec.connect())
df_Location

,Block,Lot,Street_Name,Street_Suffix,Neighborhoods_Analysis,Zipcode,Location
0,0326,023,Ellis,St,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)"
1,0306,007,Geary,St,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)"
2,0595,203,Pacific,Av,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)"
3,0156,011,Pacific,Av,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)"
4,0342,001,Market,St,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)"
...,...,...,...,...,...,...,...
198895,0113,017A,Montgomery,St,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)"
198896,0271,014,Bush,St,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)"
198897,4318,019,Indiana,St,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)"
198898,0298,029,Sutter,St,Sunset/Parkside,94116.0,"(37.74976582284573, -122.48824447038214)"


In [46]:
df_Permit=pd.read_sql_query("""SELECT Permit_Number, Permit_Type, Permit_Type_Definition From Permit ; """ , con=conec.connect())
df_Permit

,Permit_Number,Permit_Type,Permit_Type_Definition
0,201505065519,4,sign - erect
1,201604195146,4,sign - erect
2,201605278609,3,additions alterations or repairs
3,201611072166,8,otc alterations permit
4,201611283529,6,demolitions
...,...,...,...
198895,M862628,8,otc alterations permit
198896,201712055595,8,otc alterations permit
198897,M863507,8,otc alterations permit
198898,M863747,8,otc alterations permit


### DataFrame de la fecha

In [52]:
fecha_inicio='2010-01-01'
fecha_fin='2020-12-31'
dataFrameDate = pd.DataFrame({'date': pd.date_range(fecha_inicio, fecha_fin)})
dataFrameDate['year'] = dataFrameDate.date.dt.year
dataFrameDate['month'] = dataFrameDate.date.dt.month
dataFrameDate['day'] = dataFrameDate.date.dt.day
dataFrameDate['dayname'] = dataFrameDate.date.dt.day_name()
dataFrameDate['dayweek'] = dataFrameDate.date.dt.dayofweek
dataFrameDate['week'] = dataFrameDate.date.dt.weekofyear
dataFrameDate['trimester'] = dataFrameDate.date.dt.quarter
dataFrameDate = dataFrameDate[['date', 'year', 'month', 'day', 'dayname', 'dayweek', 'week', 'trimester']] 
dataFrameDate

C:\Users\Joelv\AppData\Local\Temp/ipykernel_15228/3185902291.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataFrameDate['week'] = dataFrameDate.date.dt.weekofyear


,date,year,month,day,dayname,dayweek,week,trimester
0,2010-01-01,2010,1,1,Friday,4,53,1
1,2010-01-02,2010,1,2,Saturday,5,53,1
2,2010-01-03,2010,1,3,Sunday,6,53,1
3,2010-01-04,2010,1,4,Monday,0,1,1
4,2010-01-05,2010,1,5,Tuesday,1,1,1
...,...,...,...,...,...,...,...,...
4013,2020-12-27,2020,12,27,Sunday,6,52,4
4014,2020-12-28,2020,12,28,Monday,0,53,4
4015,2020-12-29,2020,12,29,Tuesday,1,53,4
4016,2020-12-30,2020,12,30,Wednesday,2,53,4


## Carga al Data warehouse

### Cargo el dataframe de la fecha a la base de datos relacional para poder relacionar la fecha del dataframa con al fecha de los datos de la base de datos relacional para posteriormente cargarlos al data warehouse

In [59]:
dataFrameDate.to_sql(name='Dim_Time', con=conec, if_exists='append',index=False)

### Realizó el correspondiente query para poder relacionar la fecha del dataframe con la fecha de la base de datos relacional, en este caso se relacionará la fecha de Field_Date para poder relacionarla con la fecha del dataframe de la fecha y cargarla a la tabla de hechos

In [63]:

df_TablaHechos=pd.read_sql_query("""SELECT Building.id_Building as id_Building, [Location].id_Location as id_Location, Permit.id_Permit as id_Permit ,Dim_Time.TimeId as id_Time , Building.Estimated_Cost as Estimated_Cost, Building.Revised_Cost as Revised_Cost, Building.Existing_Units as Existing_Units , (Building.Estimated_Cost- Building.Revised_Cost) as Total  FROM [Location]
INNER JOIN Building
ON [Location].id_Location = Building.id_Location
INNER JOIN Permit
On Permit.id_Building = Building.id_Building
INNER JOIN Dim_Time
ON Building.Field_Date = Dim_Time.Date """ , con=conec.connect())
df_TablaHechos

,id_Building,id_Location,id_Permit,id_Time,Estimated_Cost,Revised_Cost,Existing_Units,Total
0,1,1,1,1952,4000.0,4000.0,143.0,0.0
1,2,2,2,2301,1.0,500.0,143.0,-499.0
2,3,3,3,2339,20000.0,500.0,39.0,19500.0
3,4,4,4,2503,2000.0,2000.0,1.0,0.0
4,5,5,5,2524,100000.0,100000.0,1.0,0.0
...,...,...,...,...,...,...,...,...
198895,131154,131154,131154,2395,3570.0,3570.0,0.0,0.0
198896,131155,131155,131155,2395,80000.0,0.0,7.0,80000.0
198897,131156,131156,131156,2395,80000.0,1.0,7.0,79999.0
198898,131157,131157,131157,2395,1.0,1.0,3.0,0.0


In [62]:
df_Building.to_sql(name='Dim_Building', con=conecDW, if_exists='append',index=False)
dataFrameDate.to_sql(name='Dim_Time', con=conecDW, if_exists='append',index=False)
df_Permit.to_sql(name='Dim_Permit', con=conecDW, if_exists='append',index=False)
df_TablaHechos.to_sql(name='Fact_Building', con=conecDW, if_exists='append',index=False)

## Querys

In [64]:
Consulta1=pd.read_sql_query("""select Dim_Building.Description as "Descripcion",Dim_Building.Current_Status as "Estado", Dim_Permit.Permit_Number as "Número de permiso",
Dim_Permit.Permit_Type as "Tipo de permiso"
from Dim_Permit inner join Fact_Building 
on Dim_Permit.id_Permit = Fact_Building.id_Permit inner join Dim_Building
on Fact_Building.id_Building = Dim_Building.id_Building
ORDER BY Dim_Building.Description  """ , con=conecDW.connect())
Consulta1

,Descripcion,Estado,Número de permiso,Tipo de permiso
0,--,filed,M421087,8
1,--,filed,M421087,8
2,"- add new ada restroom , break room, utility r...",issued,201710201853,8
3,"- add new ada restroom , break room, utility r...",issued,201710201853,8
4,"- attach (1) 97""w x 55"" tall mesh banner to ex...",issued,201701056601,4
...,...,...,...,...
397795,xint. remodel of 1st fl. residence. excavtion ...,complete,201507161701,8
397796,yard area: install outdoor kitchenette with gr...,filed,201507131263,8
397797,yard area: install outdoor kitchenette with gr...,filed,201507131263,8
397798,yearly roof maintance & parking . coding any b...,issued,201704214636,8


In [67]:
Consulta2=pd.read_sql_query("""select Dim_Building.Description as "Descripcion",Dim_Building.Current_Status as "Estado"
, Dim_Location.Block, Dim_Location.Street_Name, Dim_Location.Zipcode, Dim_Location.Location
from Dim_Building inner join Fact_Building
on Dim_Building.id_Building = Fact_Building.id_Building inner join Dim_Location
on Fact_Building.id_Location = Dim_Location.id_Location""" , con=conecDW.connect())
Consulta2

,Descripcion,Estado,Block,Street_Name,Zipcode,Location
0,street space,issued,3541,Sanchez,94114.0,"(37.767349757757145, -122.43140682973024)"
1,street space,issued,0829,Fell,94102.0,"(37.774914087334544, -122.42886839796792)"
2,street space,issued,6537,25th,94114.0,"(37.75006779619502, -122.43127350268124)"
3,remove and replace landscape wall,filed,6589,27th,94131.0,"(37.7462205625642, -122.4353484860549)"
4,street space,issued,1049,Pine,94115.0,"(37.78676200041077, -122.44124093333335)"
...,...,...,...,...,...,...
397795,street space,issued,0113,Montgomery,94116.0,"(37.74976582284573, -122.48824447038214)"
397796,fire alarm upgrade ref 201704123852,issued,0271,Bush,94116.0,"(37.74976582284573, -122.48824447038214)"
397797,street space,issued,4318,Indiana,94116.0,"(37.74976582284573, -122.48824447038214)"
397798,street space permit,issued,0298,Sutter,94116.0,"(37.74976582284573, -122.48824447038214)"


## Gráfica

In [ ]:
columna1=Consulta2['Descripcion']
columna1
columna2=Consulta2['Estado']
plt.figure(figsize=(10,10))
plt.barh(columna1, columna2, color="green")
## Legenda en el eje y
plt.ylabel('Descripcion ')
## Legenda en el eje x
plt.xlabel('Estado')
plt.title('Datos de la ubicación donde se encuentra la contrucción')
plt.savefig('ProductName_CantidadVendida.png')
plt.show()